Dados do Banco Mundial segmentados por Governo
===

Seja bem vindo! Esse notebook contém código para consultar e visualizar dados da base do Banco Mundial no tempo, indicando qual era o governo da época. 

Você pode acessar um preview com os gráficos funcionando (o Github não suporta conteúdo com Javascript) pelo [nbviewer](https://nbviewer.org/github/vhogemann/Notebooks/blob/main/World-Bank.ipynb#). Ou então pode ter acesso a uma versão interativa usando o serviço do Binder pelo link abaixo:

 [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/vhogemann/Notebooks/main?labpath=World-Bank.ipynb)

Para executar esse notebook na sua máquina eu recomendo utilizar o [Visual Studio Code](https://code.visualstudio.com/), com a extensão [Polyglot Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode#:~:text=To%20create%20a%20new%20polyglot,N%20key%20co) e [.NET 6.0+](https://dotnet.microsoft.com/en-us/download/dotnet/7.0).

## Sobre esse Notebook

Esse notebook foi escrito usando F#, que é uma linguagem multi-paradigma que roda na plataforma .NET da Microsoft. Os dados do Banco Mundial são acessados utilizando a excelente biblioteca [FSharp.Data](https://fsprojects.github.io/FSharp.Data/), mas você pode usar o [Google Public Data Explorer](https://www.google.com/publicdata/explore?ds=d5bncppjof8f9_) pra navegar pelos dados sem ter que se preocupar em lidar com código.

In [1]:
#r "nuget: XPlot.Plotly.Interactive, 4.0.6"
#r "nuget: FSharp.Data"

open FSharp.Data
open XPlot.Plotly

type Mandate = {
    name: string
    starting: int
    ending: int
}

let fhc1 = { name = "FHC 1"; starting = 1995; ending = 1999 }
let fhc2 = { name = "FHC 2"; starting = 1999; ending = 2003}
let lula1 = { name = "Lula 1"; starting = 2003; ending = 2007 }
let lula2 = { name = "Lula 2"; starting = 2007; ending = 2011 }
let dilma1 = { name = "Dilma 1"; starting = 2011; ending = 2015 }
let dilma2 = { name = "Dilma 2"; starting = 2015; ending = 2017 }
let temer = { name = "Temer"; starting = 2017; ending = 2018 }
let bolsonaro = { name = "Bolsonaro"; starting = 2018; ending = 2022 }

let mandates = [| fhc1; fhc2; lula1; lula2; dilma1; dilma2; temer; bolsonaro |]

let wb = WorldBankData.GetDataContext()

let br = wb.Countries.``Brazil``

let filter indicator starting ending  =
    seq {
            for year, value in indicator do
                yield year, value
        }
        |> Seq.filter(fun (year,_) -> year >= starting && year <= ending)
        |> Seq.toList
        |> List.unzip

let scatter indicator mandate =
    let years, values = filter indicator mandate.starting mandate.ending
    Scatter (name = mandate.name, x = years, y = values)

let chart title indicator =
    let plotter = scatter indicator
    mandates
    |> Seq.map plotter
    |> Chart.Plot
    |> Chart.WithTitle title

let candle indicator =
    let convert (_, values) =
        let opening = values |> Seq.head
        let closing = values |> Seq.last
        let high = values |> Seq.max
        let low = values |> Seq.min
        opening, high, low, closing
    
    mandates
    |> Seq.map ( fun mandate ->
        let opening, high, low, closing =
            filter indicator mandate.starting mandate.ending
            |> convert
        mandate.name, opening, high, low, closing
    )
    |> Chart.Candlestick


Installed Packages FSharp.Data, 5.0.2 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
br.Indicators.``GDP growth (annual %)``
|> chart "Crescimento do PIB (% anual)" 

<!DOCTYPE html>

In [3]:
br.Indicators.``GDP per capita (constant LCU)`` |> chart "PIB per Capita"

<!DOCTYPE html>

In [4]:
br.Indicators.``GDP per capita (constant LCU)`` |> candle |> Chart.WithTitle "Variação do PIB per Capita"

<!DOCTYPE html>

In [5]:
br.Indicators.``Inflation, consumer prices (annual %)``
|> chart "Inflação, preços ao consumidor (% anual)"

<!DOCTYPE html>

In [6]:
br.Indicators.``Unemployment, total (% of total labor force) (modeled ILO estimate)``
|> chart "Desemprego"

<!DOCTYPE html>

In [7]:
br.Indicators.``Unemployment, total (% of total labor force) (modeled ILO estimate)`` 
|> candle 
|> Chart.WithTitle "Desemprego % Total da força de trabalho"

<!DOCTYPE html>

In [8]:
br.Indicators.``Access to electricity (% of population)``
|> chart "Acesso a Eletricidade"

<!DOCTYPE html>

In [10]:
br.Indicators.``Official exchange rate (LCU per US$, period average)`` |> chart "Taxa de câmbio"

<!DOCTYPE html>

In [11]:
br.Indicators.``Central government debt, total (% of GDP)`` |> chart "Dívida Pública %PIB"

<!DOCTYPE html>